## Elasticity matrix

An elasticity matrix between all the aggregated zones is constructed. Elasticities value deviate a standard elasticity, found in literature.

The deviations depend on the slices in which zones are situated.

1. Group zones based on the aggregated slices made in QGIS. In this case specifically, there is one central zone and 6 'pie slices' around it. This is to make a difference between the actual zone of interest (slice 0) and all others providing demand.

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import geopandas as gpd 
import numpy as np

city = 'BRUSSEL'
radius = '40'
combined_shapefile = gpd.read_file(f'data_map/QGIS/{city}_{radius}_10_aggr_comb.shp')
# split shapefile into pie slices:
slice_0 = gpd.read_file(f'data_map/QGIS/slices/slice_0.shp')
slice_1 = gpd.read_file(f'data_map/QGIS/slices/slice_1.shp')
slice_2 = gpd.read_file(f'data_map/QGIS/slices/slice_2.shp')
slice_3 = gpd.read_file(f'data_map/QGIS/slices/slice_3.shp')
slice_4 = gpd.read_file(f'data_map/QGIS/slices/slice_4.shp')
slice_5 = gpd.read_file(f'data_map/QGIS/slices/slice_5.shp')
slice_6 = gpd.read_file(f'data_map/QGIS/slices/slice_6.shp')
slices = [slice_0, slice_1, slice_2, slice_3, slice_4, slice_5, slice_6]

# initialize slice list
slice_list = [0 for i in range(len(combined_shapefile))]
i = 1

for slice in slices[1:]:
    clusters = slice['cluster'].values
    for cluster in combined_shapefile['cluster']:
        if cluster in clusters:
            print(cluster)
            slice_list[cluster] = i
    i += 1

combined_shapefile['slice'] = slice_list
combined_shapefile.to_file(f'data_map/QGIS/{city}_{radius}_10_aggr_comb.shp')

2. Initialise elasticity matrix.

In [ ]:
# initialize elasticity value matrix with ones 
standard_elasticity = 1

standard_elasticity_matrix = [[standard_elasticity for i in range(len(slices))] for j in range(len(slices))]
standard_elasticity_matrix

3. Index all values in the elasticity matrix based on relations between slices (elasticities found in literature, these should normally be thoroughly researched for specific use-cases). Be aware that this is at the moment hardcoded, but it can be improved to have a more flexible style later on.

In [ ]:
# indexing: standard_elasticity_matrix[row][column]
# e.g. standard_elasticity_matrix[0][4] means row 0, column 4. Interpretation: from slice 0 to slice 4

to_own = 0.18
to_neighbour = 0.18
to_others = 0.15
to_opposite = 0.15
to_Brussels = 0.22
from_Brussels = 0.22
in_Brussels = 0.25


# from 0 to 0
standard_elasticity_matrix[0][0] *= in_Brussels

# from i to i
# from i to Brussels
# from Brussels to i
for i in range(1,7):
    standard_elasticity_matrix[i][i] *= to_own
    standard_elasticity_matrix[0][i] *= from_Brussels
    standard_elasticity_matrix[i][0] *= to_Brussels


# from 1 to neighbouring slices
standard_elasticity_matrix[1][2] *= to_neighbour
standard_elasticity_matrix[1][6] *= to_neighbour
# from 1 to opposite slice
standard_elasticity_matrix[1][4] *= to_opposite
# from 1 to other slices
standard_elasticity_matrix[1][3] *= to_others
standard_elasticity_matrix[1][5] *= to_others


# from 2 to neighbouring slices
standard_elasticity_matrix[2][1] *= to_neighbour
standard_elasticity_matrix[2][3] *= to_neighbour
# from 2 to opposite slice
standard_elasticity_matrix[2][5] *= to_opposite
# from 2 to other slices
standard_elasticity_matrix[2][4] *= to_others
standard_elasticity_matrix[2][6] *= to_others


# from 3 to neighbouring slices
standard_elasticity_matrix[3][2] *= to_neighbour
standard_elasticity_matrix[3][4] *= to_neighbour
# from 3 to opposite slice
standard_elasticity_matrix[3][6] *= to_opposite
# from 3 to other slices
standard_elasticity_matrix[3][1] *= to_others
standard_elasticity_matrix[3][5] *= to_others


# from 4 to neighbouring slices
standard_elasticity_matrix[4][3] *= to_neighbour
standard_elasticity_matrix[4][5] *= to_neighbour
# from 4 to opposite slice
standard_elasticity_matrix[4][1] *= to_opposite
# from 4 to other slices
standard_elasticity_matrix[4][2] *= to_others
standard_elasticity_matrix[4][6] *= to_others


# from 5 to neighbouring slices
standard_elasticity_matrix[5][4] *= to_neighbour
standard_elasticity_matrix[5][6] *= to_neighbour
# from 5 to opposite slice
standard_elasticity_matrix[5][2] *= to_opposite
# from 5 to other slices
standard_elasticity_matrix[5][1] *= to_others
standard_elasticity_matrix[5][3] *= to_others

# from 6 to neighbouring slices
standard_elasticity_matrix[6][5] *= to_neighbour
standard_elasticity_matrix[6][1] *= to_neighbour
# from 6 to opposite slice
standard_elasticity_matrix[6][3] *= to_opposite
# from 6 to other slices
standard_elasticity_matrix[6][2] *= to_others
standard_elasticity_matrix[6][4] *= to_others

# round results up to 3 
slice_elasticity_matrix =  [[round(standard_elasticity_matrix[i][j], 3) for i in range(len(slices))] for j in range(len(slices))]

slice_elasticity_matrix


4. Make a zone elasticity matrix (600x600) using the slice elasticity matrix (7x7).

In [ ]:
combined_shapefile = gpd.read_file(f'data_map/QGIS/{city}_{radius}_10_aggr_comb.shp')
zone_elasticity_matrix = np.zeros((len(combined_shapefile), len(combined_shapefile)))

# Loop through all pairs of observations and fill in the matrix with the corresponding slice value
for i in range(len(combined_shapefile)):
    for j in range(len(combined_shapefile)):
        k = combined_shapefile.iloc[i]['slice']
        l = combined_shapefile.iloc[j]['slice']
        zone_elasticity_matrix[i][j] = slice_elasticity_matrix[k][l]

5. Determine the useful path to the elasticity matrix. This is for ease of use later on.

In [ ]:
elasticity_path = f"data_map/STA/elasticity/Brussel_{radius}"
np.savetxt(elasticity_path,zone_elasticity_matrix)

    Congratulations, you now have made your elasticity matrix B as a building block to perform elastic assignments!